<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Знакомство-с-данными" data-toc-modified-id="Знакомство-с-данными-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Знакомство с данными</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Ответы-на-вопросы" data-toc-modified-id="Ответы-на-вопросы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ответы на вопросы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


**Описание данных** 

- *children* — количество детей в семье
- *days_employed* — общий трудовой стаж в днях
- *dob_years* — возраст клиента в годах
- *education* — уровень образования клиента
- *education_id* — идентификатор уровня образования
- *family_status* — семейное положение
- *family_status_id* — идентификатор семейного положения
- *gender* — пол клиента
- *income_type* — тип занятости
- *debt* — имел ли задолженность по возврату кредитов
- *total_income* — ежемесячный доход
- *purpose* — цель получения кредита

## Знакомство с данными

Откроем файл, предоставленный кредитным отделом банка и изучим информацию в нем.

In [1]:
# открытие и запись в переменную data таблицы
import pandas as pd 

data = pd.read_csv('data.csv')
data.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

На первый взгляд выгрузка содержит мало ошибок, но в таблице почти 22 тысячи строк. Столбцы таблицы названы корректно, переименовывать их не придётся. Если присмотреться к данным повнимательнее, то возникает ряд вопросов. 
* Столбец 'days_employed' - это общий трудовой стаж в днях. Почти все числа в нём отрицательные (возможно, это строки и минус - это тире, или количество дней считали, вычитая начала стажа текущий момент, а не наоборот?), а те, что не отрицательные - очень большие. Например, если человек работал 340266 дней все 365 дней в году, значит его стаж - почти 932 года! Это маловероятно, а точнее невероятно. Возможно, это указан стаж в часах? Тогда его стаж составит примерно 38 лет. Это уже ближе к вероятному, но для 53-летнего человека всё равно многовато. Также видно, что большие значение указаны там, где income_type это 'пенсионер'
* Столбец 'total_income' — это ежемесячный доход. В каких единицах измерения? В рублях? В йенах? В цветочках? Допустим, в рублях.

## Предобработка данных

### Обработка пропусков

Сначала просмотрим общую информацию о таблице.

In [2]:
# просмотр общей информации
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Видно, что пропуски есть в столбцах days_employed и total_income. Но нельзя исключать, что пропуски отсутствуют и в других столбцах, ведь они могут быть записаны, как строки или числовые значения. Начнем сначала. Столбец с количеством детей содержит целочисленный тип, поэтому строк там точно нет. Проверим уникальные значения. 

In [3]:
# уникальные значения количества детей в таблице
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Всё бы ничего, 20 детей ещё куда не шло (возможно, кто-то случайно приписал 0), но минус 1 - это странно. Тут могла бы быть шутка про сдачу ребёнка в детдом, но я не буду её шутить. Возможно, клиенты, у которых нет детей, просто пропустили этот вопрос и по умолчанию значение стоит -1. 

In [4]:
# количество клиентов со значением 'children' равным -1
data[data['children'] < 0]['children'].count()

47

Так как одной из основных задач является нахождение влияния наличия детей на факт погашения кредита, проверим столбец 'debt' у этих клиентов.

In [5]:
# сколько среди таких клиентов должников
data[data['children'] < 0]['debt'].value_counts()

0    46
1     1
Name: debt, dtype: int64

Почти все, за исключением одного пользователя, не имели задолженностей по кредиту, поэтому если мы самостоятельно установим значения 1 или 0 в таких строках, то мы искуственно повлияем на результат. Это достаточно маленький процент записей из всей таблицы, поэтому мы можем просто исключить эти строки из исследования.  

In [6]:
# удаляем строки и проверяем результат
data = data.loc[data['children'] >= 0, :] 
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21478 entries, 0 to 21477
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21478 non-null  int64  
 1   days_employed     19307 non-null  float64
 2   dob_years         21478 non-null  int64  
 3   education         21478 non-null  object 
 4   education_id      21478 non-null  int64  
 5   family_status     21478 non-null  object 
 6   family_status_id  21478 non-null  int64  
 7   gender            21478 non-null  object 
 8   income_type       21478 non-null  object 
 9   debt              21478 non-null  int64  
 10  total_income      19307 non-null  float64
 11  purpose           21478 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Строки удалились, проверяем дальше. Посмотрим, все ли клиенты в этой выгрузке имеют возраст.

In [7]:
# сортируем количество людей опредлённого возраста по его возрастанию
data['dob_years'].value_counts().sort_index().head()

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64

101 человек с нулевым возрастом - это грустно. Посмотрим, какие у них другие значения в таблице.

In [8]:
# выводим информацию о клиентах с нулевым возрастом
data[data['dob_years'] == 0].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
577,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1035,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


Даже по стажу можно сказать, что пропущенный возраст может быть абсолютно разным, поэтому просто заменить на средний возраст тут не получится. Посмотрим, как эти новорожденные выплачивают кредиты.

In [9]:
# сколько среди таких клиентов должников
data[data['dob_years'] == 0]['debt'].value_counts()

0    93
1     8
Name: debt, dtype: int64

Только 8% не выплачивают кредит вовремя. И 101 человек - это тоже достаточно маленький процент от всей выгрузки, поэтому думаю исключение этих строк не очень повлияет на результат. 

In [10]:
# проверяем, удалились ли строки
data = data.loc[data['dob_years'] > 0, :] 
data.reset_index(drop=True, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     19216 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      19216 non-null  float64
 11  purpose           21377 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски успешно удалились. Видно, что пропуски остались в столбцах days_employed и total_income, причем их одинаковое количество. Возможно, они даже содержаться в одних и тех же записях. Проверим это.

In [11]:
# проверка количества пропусков в столбцах
if data['days_employed'].isna().sum() == data['total_income'].isna().sum():
    print('Количество пропусков в столбцах days_employed и total_income совпадает')

Количество пропусков в столбцах days_employed и total_income совпадает


In [12]:
# вывод на экран таблицы с пропусками
data_nan = data[data['days_employed'].isna()]
data_nan.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Возможно, пропуски и правда содержатся в одних и тех же записях. 

In [13]:
# проверка пропусков 
if data_nan['days_employed'].isna().sum() == data_nan['total_income'].isna().sum():
    print('Пропуски в столбцах days_employed и total_income содержаться в одних и тех же записях.')

Пропуски в столбцах days_employed и total_income содержаться в одних и тех же записях.


Значения зарплаты отсутствуют тогда и только тогда, когда отсутствуют сведения о стаже. Возможно, в таблице указывался стаж по трудовой книжке, поэтому значения могут быть пропущены. Соответственно, и зарплата указана только там, где стаж есть. 

Проверим, может быть таких случаев немного и мы сможем их просто исключить из исследования.

In [14]:
# расчёт процента строк с пропусками
print('Процент строк, содержащих пропуски: {:.0%}'.format(data_nan['children'].count() / data['children'].count()))

Процент строк, содержащих пропуски: 10%


10% строк с пропусками - это довольно большой процент, поэтому просто удалить эти строки нельзя, ведь мы потеряем существенное количество важной информации. Основной задачей исследования является обнаружение влияния семейного положения и количества детей на факт погашения клиентом кредита. Эта информация представлена в таблице полностью. Даже по первым строкам таблицы с пропусками видно, что отсутсвие информации о стаже и заработной плате клиента не влияет на наличие значения в столбце о наличии у него задолженностей по кредиту. 

Значения в столцах с пропусками являются количественными характеристиками, поэтому мы можем заменить эти значения нулями, средним арифметическим или медианой. 

Но предже чем мы это сделаем, разберемся c отрицательными и огромными значениям в столбце 'days_employed'.

In [15]:
# поиск минимального и максимального положительного числа в стоблце
max_positive = data['days_employed'].max()
print('Максимальное положительное значение:', max_positive)
min_positive = data['days_employed'][data['days_employed'] >= 0].min()
print('Максимальное положительное значение:', min_positive)

Максимальное положительное значение: 401755.40047533
Максимальное положительное значение: 328728.72060451825


Максимальное и минимальное положительное значения довольно велики, значит все положительные значения указаны некорректно. Проверим гипотезу о том, что такие значения установлены только у пенсионеров. 

In [16]:
# уникальные значения в столбце 'income_type', где зарплата больше минимального положительного значения
max_money_income_type_unique = data[data['days_employed'] >= min_positive]['income_type'].unique()
max_money_income_type_unique

array(['пенсионер', 'безработный'], dtype=object)

Гипотеза не подтвердилась, огромный стаж установлен и для безработных.

Проверим, установлен ли этот стаж в часах. Разденим каждое такое значение на количество часов в году и сравним с возрастом клиента. 

In [17]:
# проверка корректности данных:
# из возраста вычитаем стаж, рассчитаный в годах для тех, у кого стаж указан положительный
years_not_employed = data[data['days_employed'] >= min_positive]['dob_years'] - data[data['days_employed'] >= min_positive]['days_employed'] / 365 / 24
years_not_employed[years_not_employed <= 0].sort_values().head()

19299   -18.451731
12413   -16.321016
1230    -16.215098
13849   -16.016505
16046   -15.592260
dtype: float64

Что-то явно не сходится. Посмотрим подробную информацию на подобных клиентах.

In [18]:
# проверка конкретного пользователя
data.loc[19299][:]

children                               0
days_employed              389397.167577
dob_years                             26
education                         высшее
education_id                           0
family_status            женат / замужем
family_status_id                       0
gender                                 F
income_type                    пенсионер
debt                                   0
total_income               214963.301941
purpose             покупка недвижимости
Name: 19299, dtype: object

Странно - пенсионер в возрасте 26 лет. Возраст здесь явно указан неверно. Почему? "Человеческий фактор" :) Посмотрим, сколько у нас таких пенсионеров. 

In [19]:
# сортировка количества пенсионеров определённого возраста по его возрастанию
data[data['income_type'] == 'пенсионер']['dob_years'].value_counts().sort_index().head()

22    1
24    1
26    2
27    3
31    1
Name: dob_years, dtype: int64

Я думаю, что проблема данных именно в некорректном заполнении возраста, поскольку прежде мы установили, что большой стаж установлен только для пенсионеров и безработных. Поэтому некорректно будет сравнивать рассчитаный стаж с возрастом. Я бы вообще не доверяла этому столбцу, поскольку мы точно не можем исправить в нем возраст на истинный. Если далее в рассчетах понадобится информация именно о возрасте, то будем использовать эти значения "аккуратно". 

Заменим эти огромные значения на близкие к истинне, разделив их на количество часов в дне. 

In [20]:
# приведение аномальных значений стажа к более реальному:
# делим существующие значения на количество часов в дне
data.loc[data['days_employed'] >= min_positive, 'days_employed'] = data[data['days_employed'] >= min_positive]['days_employed'] / 24
print('Максимальное положительное значение:',data['days_employed'].max())

Максимальное положительное значение: 16739.80835313875


После замены значений максимальный стаж в днях уже не такой уж и запредельный. Теперь разберёмся с отрицательными значениями. Заменим их на положительные. 

In [21]:
# замена всех отричательных значений стажа на положительные 
data['days_employed'] = abs(data['days_employed'])
print('Минимальное значение стажа - {:.2f}, максимальное - {:.2f}'.format(data['days_employed'].min(), data['days_employed'].max()))

Минимальное значение стажа - 24.14, максимальное - 18388.95


Минимальное и максимальное значения стажа теперь приемлемы.

Вернемся к пропущенным значениям. Я думаю, надо ли вообще их заменять, ведь NaN - это float тип, который может участвовать в арифметических операциях? Замена на приближённые значения могут повлиять на результат. Но я удалять такие строки тоже не стоит, ведь их почти 10% и они содержат значимую информацию в других столбцах. Заменим эти значения на медианные в разрезе типа занятости, так, на мой взгляд, будет правдоподобнее всего. 

In [22]:
# группируем столбцы 'days_employed' и 'total_income' по значениям 'income_type'
# ищем медиану сгруппированных значений
# добавляем результат вместо соответствующих пропущенных значений
data['days_employed'] = data['days_employed'].fillna(data.groupby('income_type')['days_employed'].transform('median'))
data['total_income'] = data['total_income'].fillna(data.groupby('income_type')['total_income'].transform('median'))

Проверим, успешно ли заполнены пропуски в таблице.

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     21377 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21377 non-null  float64
 11  purpose           21377 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

На этапе обработки пропусков были приняты следующие решения: 
* Данные с нулевым возрастом клиента и наличием -1 ребенка были удалены из датасета: количество таких строк не велико (около 1%), поэтому это не отразиться на результатах значительно.
* Данные стажа были обработаны в соответствии со здравым смыслом. 
* Пропуски, обнаруженные в столбцах стажа и зарплаты, оказались в одних и тех же строчках, но таких строчек около 10%. Столько строчек удалять из выгрузки не стоит, к тому же в других ячейках они содержат важную информацию, которую терять в таком количестве нельзя. Поэтому было решено заполнить эти пропуски медианными значениями в разрезе типа занятости. 

Проведённая предобработка данных может минимально исказить результаты исследования, и не стоит об этом забывать. 

### Замена типа данных

Взглянем ещё раз на информацию о таблице и обратим внимание на типы данных.

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21377 non-null  int64  
 1   days_employed     21377 non-null  float64
 2   dob_years         21377 non-null  int64  
 3   education         21377 non-null  object 
 4   education_id      21377 non-null  int64  
 5   family_status     21377 non-null  object 
 6   family_status_id  21377 non-null  int64  
 7   gender            21377 non-null  object 
 8   income_type       21377 non-null  object 
 9   debt              21377 non-null  int64  
 10  total_income      21377 non-null  float64
 11  purpose           21377 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


На самом деле с типами данных тут всё в порядке на мой взгляд, обязательной замены тут не требуется. Но можно изменить столбцы 'days_employed' и 'total_income' на целочисленный тип, потому что вряд ли нам сильно понадобится дробная часть дней стажа или рублей. 

In [25]:
# замена типа столбцов на целочисленный
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [26]:
# просмотр таблицы
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Вещественный тип данных в двух столбцах таблицы был изменен на целочисленный тип, но это не было обязательным действием, поскольку вещественный тип никак не помешал бы рассчётам.

### Обработка дубликатов

Прежде чем искать дубликаты, посмотрим на уникальные значения в столбцах 'education', 'family_status' и 'income_type'. Возможно, значения записаны в разных регистрах. Со значениями в столбце 'purpose' разберёмся позже, когда будем лемматизировать. 

Начнём с образования.

In [27]:
# количество различных значений в столбце образования
data['education'].value_counts()

среднее                13660
высшее                  4678
СРЕДНЕЕ                  766
Среднее                  706
неоконченное высшее      665
ВЫСШЕЕ                   272
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Почти все типы образования записаны в различных регистрах. Исправим это переводом всех значений в нижний регистр.

In [28]:
# понижение регистра значений в столбце образования
data['education'] = data['education'].str.lower()

Проверим, всё ли получилось.

In [29]:
# количество различных значений в столбце образования
data['education'].value_counts()

среднее                15132
высшее                  5216
неоконченное высшее      741
начальное                282
ученая степень             6
Name: education, dtype: int64

Отлично, теперь все 5 типов образования записаны в едином виде. 

Теперь рассмотрим значения в стоблце о семейном положении

In [30]:
# количество различных значений в столбце о семейном образовании
data['family_status'].value_counts()

женат / замужем          12302
гражданский брак          4151
Не женат / не замужем     2792
в разводе                 1181
вдовец / вдова             951
Name: family_status, dtype: int64

Здесь всё в порядке, теперь проверим 'income_type'

In [31]:
# количество различных значений в столбце о семейном образовании
data['income_type'].value_counts()

сотрудник          11038
компаньон           5056
пенсионер           3828
госслужащий         1449
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Со столбцами всё в порядке, теперь можем приступать к поиску и удалению дубликатов. На самом деле странно, что в таблице даже нет id пользователей, как без этого определить, что строчки принадлежат одному и тому же клиенту? Исключим вероятность полного совпадения записей разных людей и будем считать, что соответствующие дубликаты содержат информацию об одном и том же человеке.

In [32]:
# количество строчек, у которых есть дубликат
data[data.duplicated()]['days_employed'].count()

71

71 строчка с дубликатами. Появление дубликатов можно объяснить тем, что человек мог оформить заявку на кредит через сайт банка, а потом прийти в банк, где в базу внесут те же самые данные. Удалим дубликаты, они нам не нужны.

In [33]:
# удаляем дубликаты из таблицы и обновляем индексацию
data = data.drop_duplicates().reset_index(drop=True)
# проверка на оставшиеся дубликаты
data[data.duplicated()]['days_employed'].count()

0

**Вывод**

Все явные дубликаты удалены, но мы не проанализировали значения в последнем столбце, ведь строки в нем различные, но написанные по разному цели получения кредита на деле могут значить одно и тоже. В решении этой проблемы нам поможет лемматизация.

### Категоризация данных

Имеет смысл разделить на категории доход. Выделим три категории дохода: 
* низкий - до 100000
* средний - от 100000 до 200000
* высокий - от 200000 

Интервалы выбраны исходя из моего личного представления о данных в столбце дохода. Возможно, можно применить другие способы определения интервалов, но я пока не владею ими.

In [34]:
# функция определения категории дохода
def income_category(income):
    if income < 100000:
        return 'низкий'
    elif income < 200000:
        return 'средний'
    else:
        return 'высокий'
    
# создаем новый столбец и заполняем его результатом вызова функции к каждому значению столбца 'total_income'
data['income_category'] = data['total_income'].apply(income_category)

In [35]:
# распределение категорий
data['income_category'].value_counts()

средний    11841
высокий     5033
низкий      4432
Name: income_category, dtype: int64

В основном у клиентов средний доход, что логично, а высокого и низкого - поровну. Значит границы определены верно.

In [36]:
# посмотрим, что получилось
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,средний
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,средний


Теперь цели кредита зароботная плата категоризированы, так будет удобнее анализировать влияние цели взятья кредита на факт его погашения.

**Вывод**

Категоризация зарплаты поможет проанализировать влияние именно уровня дохода на факт выплаты кредита. Можно было бы удалить информацию об образовании и семейном положении и оставить только их id, но в данном случае я не думаю, что они сильно мешают. Будет проще запутаться в нескольких таблицах с названиями и их индексами. Но это конкретно в этом случае. 

## Ответы на вопросы

Чтобы ответить на следующие вопросы, воспользуемся сводными таблицами. Они позволят объединить и обработать данные и представить результат в удобном для восприятия виде.

In [37]:
# функция для составления сводной таблицы
def pivot_on_groups(df, feature):
    return df.groupby(feature)['debt'].agg(['count', 'sum', lambda x: '{:.2%} '.format(x.mean())])    

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [38]:
# сводная таблица в разрезе количества детей
pivot_on_groups(data, 'children')

,count,sum,<lambda_0>
children,,,
0,14022,1058,7.55%
1,4792,441,9.20%
2,2039,194,9.51%
3,328,27,8.23%
4,41,4,9.76%
5,9,0,0.00%
20,75,8,10.67%


In [39]:
# сводная таблица по бинарной классификации
pivot_on_groups(data, data['children'].apply(lambda x: 'Есть' if x>0 else 'Нет'))

,count,sum,<lambda_0>
children,,,
Есть,7284,674,9.25%
Нет,14022,1058,7.55%


**Вывод**

С ростом числа детей в семье процент людей, имеющих долги по кредиту, растет. Так же падает количество клиентов. Значит количество детей и правда влияет на факт погашения кредита в срок. Гипотеза подтвердилась. К тому же клиенты, у которых нет детей чаще погашают кредит вовремя.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
# создание сводной таблицы в разрезе семейного положения
pivot_on_groups(data, 'family_status')

,count,sum,<lambda_0>
family_status,,,
Не женат / не замужем,2789,273,9.79%
в разводе,1181,85,7.20%
вдовец / вдова,950,62,6.53%
гражданский брак,4125,386,9.36%
женат / замужем,12261,926,7.55%


**Вывод**

Клиенты, никогда не состоящие в официальном браке, чаще остальных имеют задолженность по кредиту. Значит семейное положение влияет на погашение кредита в срок. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
# создание сводной таблицы относительно уровня дохода
pivot_on_groups(data, 'income_category')

,count,sum,<lambda_0>
income_category,,,
высокий,5033,357,7.09%
низкий,4432,353,7.96%
средний,11841,1022,8.63%


**Вывод**

Нельзя точно сказать, что чем выше у клиента зарплата, тем вероятнее он закроет кредит в срок. Возможно, если бы мы категоризировали доходы клиентов по другим интервалам, то результат был бы иной. В нашем случае с ростом дохода вероятность долгов по кредиту изменяется не монотонно. У среднего класса долги возникают чаще. Люди с низким доходом понимают, что не потянут еще и процент за просрочку, поэтому отказывают себе в чем-то, чтобы отложить деньги на выплату кредита. Богатые люди, как правило, более финансово грамотные и кредиты обычно берут на бизнес. А вот средний класс может позволить себе задержку, ведь *все равно через неделю зарплата*.

## Общий вывод

Начальная гипотеза о том, что количество детей и семейное положение влияет на факт погашения кредита в срок подтвердилась: чем больше у клиента детей, тем вероятнее он будет иметь долг по кредиту, а клиенты, состоящие когда-либо в официальном браке, вероятнее всего выплатят кредит в срок. 

Предобработка данных влючала в себя обработку артефактов и пропущенных или некорректных значений, что могло минимально повлиять на получившийся результат. Также были удалены дубликаты, а для категоризации целей кредита проведена их лемматизация, после чего было выделено пять основных категорий целей кредита. Классификация заработной платы я считаю субъективной, поэтому на вопрос о влиянии уровня зарабоной платы на факт погашения кредита ответить однозначно не получилось.

Подводя итог, можно выделить самые надёжные и ненадёжные категории граждан в каждой группе.

Факторы, показывающие что клиент с большей вероятностью закроет кредит в срок (с указанием процента невозврата): 
* отсутствие детей - 7.55%
* семейный статус "в разводе" - 7.19%, или "вдова/вдовец" - 6.52% 
* высокий уровнь заработной платы - 7.09%

Факторы, влияющие на возможное наличие долгов по кредиту: 
* многодетные семьи (например, 4 ребёнка - 9.76%)
* семейный статус не женат/не замужем - 9.78%, или гражданский брак	9.35%
* средний уровень заработной платы - 8.63%
